In [1]:
import os
import json
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from utils.gemini_service import GeminiModel, GeminiJsonEngine, GeminiSimpleChatEngine
from utils.pydantic_service import pydantic_schema_to_tool_format



os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/debasmitroy/Desktop/programming/gemini-agent-assist/key.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "hackathon0-project"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

/Users/debasmitroy/Desktop/programming/gemini-agent-assist/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Model Definition

In [2]:
model = GeminiModel(model_name="gemini-1.5-flash-002", temperature=0.8)

Initialized GeminiModel with model gemini-1.5-flash-002 , project hackathon0-project, location us-central1


## Generate Plain Text 

In [3]:
content_roles = [
    {"role": "user", "content": "You are an expert assistant."},
    {"role": "model", "content": "Thank you for the compliment."},
    {"role": "user", "content": "What can I help you with today?"}
]
systemInstructions = "You are an AI assistant. You can help the user with their queries."
response = model.generate_content(content_roles, systemInstructions, simplify_output=True)
response

"That's a bit cheeky!  As an expert AI assistant, I'm here to help *you*.  Tell me what you need assistance with.  The more detail you provide, the better I can assist you.  For example, are you looking for information on a specific topic, help with a problem, assistance with writing something, or something else entirely?\n"

## Engine Wrapper

In [2]:
gemini_simple_chat_engine = GeminiSimpleChatEngine(model_name="gemini-1.5-flash-002", temperature=0.8, max_output_tokens=100, systemInstructions="You are an expert assistant. What can I help you with today?")

Initialized GeminiModel with model gemini-1.5-flash-002 , project hackathon0-project, location us-central1


In [3]:
response = gemini_simple_chat_engine("What is the capital of France?")

In [4]:
response

'The capital of France is Paris.\n'

## Gemini Structured Output

In [5]:
# Sample request for function calling
content_roles = [
    {"role": "user", "content": "what is 12+34-56?"}
]
# systemInstructions = "You are an AI assistant. You are helping the user with their math homework."

tools = [
    {
        "name": "addition",
        "description": "Add multiple numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "numbers": {
                    "type": "array",
                    "description": "A list of numbers to add",
                    "items": {
                        "type": "number"
                    }
                }
            },
            "required": ["numbers"]
        }
    },
    {
        "name": "subtraction",
        "description": "Subtract two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "minuend": {
                    "type": "number",
                    "description": "The number to subtract from"
                },
                "subtrahend": {
                    "type": "number",
                    "description": "The number to subtract"
                }
            },
            "required": ["minuend", "subtrahend"]
        }
    }
]


response_func = model.generate_funccall_content(content_roles, tools, simplify_output=True)
response_func


[{'name': 'addition', 'args': {'numbers': [12, 34]}},
 {'name': 'subtraction', 'args': {'minuend': 46, 'subtrahend': 56}}]

## Gemini Strucutre Output (Cleaner Code)

In [6]:
class AdditionParams(BaseModel):
    """
    Parameters for adding multiple numbers.
    """
    numbers: List[float] = Field(..., description="A list of numbers to add")

class SubtractionParams(BaseModel):
    """
    Parameters for subtracting two numbers.
    """
    minuend: float = Field(..., description="The number to subtract from")
    subtrahend: float = Field(..., description="The number to subtract")    

In [10]:
content_roles = [
    {"role": "user", "content": "what is 12+34?"},
    {"role": "user", "content": "Now subtract 56 from the result."}
]
# systemInstructions = "You are an AI assistant. You are helping the user with their math homework."

tools = [
    pydantic_schema_to_tool_format(AdditionParams),
    pydantic_schema_to_tool_format(SubtractionParams)
]

response_func = model.generate_funccall_content(content_roles, tools, simplify_output=True)
response_func

[{'name': 'subtraction', 'args': {'minuend': 46, 'subtrahend': 56}}]

## Another Example With Pydantic

In [11]:
class SentimentAnalysisParams(BaseModel):
    """
    Parameters for sentiment analysis.
    """
    text: str = Field(..., description="The text to analyze")
    anger_level: float = Field(..., description="The anger level of the text, from 0.0 to 1.0")
    joy_level: float = Field(..., description="The joy level of the text, from 0.0 to 1.0")
    sadness_level: float = Field(..., description="The sadness level of the text, from 0.0 to 1.0")
    fear_level: float = Field(..., description="The fear level of the text, from 0.0 to 1.0")

In [12]:
content_roles=[{"role": "user", "content": "I am feeling happy today. Now do a sentiment analysis on this text."}]

In [13]:
sentiment_result = model.generate_funccall_content( 
    content_roles,
    tools=[pydantic_schema_to_tool_format(SentimentAnalysisParams)],
    simplify_output=True
)
sentiment_result

[{'name': 'sentimentanalysis',
  'args': {'anger_level': 0,
   'joy_level': 1,
   'sadness_level': 0,
   'text': 'I am feeling happy today.',
   'fear_level': 0}}]

## Another Example With Pydantic (Nested Schema)

In [14]:
class DebitCard(BaseModel):
    """
    Parameters for a debit card.
    """
    card_number: str = Field(..., description="The debit card number, xxx if not available") 
    card_holder: str = Field(..., description="The name of the card holder, xxx if not available")
    expiry_date: str = Field(..., description="The expiry date of the card, xxx if not available")
    cvv: str = Field(..., description="The CVV of the card, xxx if not available")

class CreditCard(BaseModel):
    """
    Parameters for a credit card.
    """
    card_number: str = Field(..., description="The credit card number, xxx if not available")
    card_holder: str = Field(..., description="The name of the card holder, xxx if not available")
    expiry_date: str = Field(..., description="The expiry date of the card, xxx if not available")
    cvv: str = Field(..., description="The CVV of the card, xxx if not available")
    credit_limit: float = Field(..., description="The credit limit of the card, xxx if not available")

class Person(BaseModel):
    """
    Parameters for a person.
    """
    name: str = Field(..., description="The name of the person")
    age: int = Field(..., description="The age of the person")
    debit_card_list: List[DebitCard] = Field(..., description="The list of debit cards of the person")
    credit_card_list: List[CreditCard] = Field(..., description="The list of credit cards of the person")

content_roles = [
    {"role": "user", "content": "I am John Doe. I am 30 years old. I have 2 debit cards and 1 credit card. My debit card details are: 1234 5678 9012 3456, John Doe, 12/23, 123 and 2345 6789 0123 4567, John Doe, 12/24, 456. My credit card details are: 3456 7890 1234 5678, John Doe, 12/25, 789, 10000."}
]

In [15]:
person_result = model.generate_funccall_content(
    content_roles,
    tools=[pydantic_schema_to_tool_format(Person)],
    simplify_output=True
)
person_result

[{'name': 'person',
  'args': {'name': 'John Doe',
   'age': 30,
   'credit_card_list': [{'card_holder': 'John Doe',
     'card_number': '3456 7890 1234 5678',
     'expiry_date': '12/25',
     'credit_limit': 10000,
     'cvv': '789'}],
   'debit_card_list': [{'expiry_date': '12/23',
     'card_number': '1234 5678 9012 3456',
     'cvv': '123',
     'card_holder': 'John Doe'},
    {'card_holder': 'John Doe',
     'cvv': '456',
     'expiry_date': '12/24',
     'card_number': '2345 6789 0123 4567'}]}}]

## Engine Wrapper

In [16]:
person_engine = GeminiJsonEngine(
    model_name="gemini-1.5-flash-002",
    basemodel=Person,
    temperature=0.8,
    max_output_tokens=100,
    systemInstructions="You are an AI assistant. You are helping the user with their personal information. You will be asked to extract information from the user's text."
)

Initialized GeminiModel with model gemini-1.5-flash-002 , project hackathon0-project, location us-central1


In [17]:
person_ans = person_engine(
    user_query = """
    There are two persons.
    One is John Doe. He is 30 years old. He has 2 debit cards and 1 credit card. His debit card details are: 1234 5678 9012 3456, John Doe, 12/23, 123 and 2345 6789 0123 4567, John Doe, 12/24, 456. His credit card details are: 3456 7890 1234 5678, John Doe, 12/25, 789, 10000.
    The other person is Jane Doe. She is 25 years old. She has 1 debit card and 2 credit cards. Her debit card details are: 4567 8901 2345 6789, Jane Doe, 12/24, 234. Her credit card details are: 5678 9012 3456 7890, Jane Doe, 12/26, 567, 20000 and 6789 0123 4567 8901, Jane Doe, 12/27, 678, 30000.
    """
)

In [18]:
person_ans[0] , person_ans[1]

({'name': 'John Doe',
  'debit_card_list': [{'card_holder': 'John Doe',
    'expiry_date': '12/23',
    'card_number': '1234 5678 9012 3456',
    'cvv': '123'},
   {'expiry_date': '12/24',
    'card_holder': 'John Doe',
    'cvv': '456',
    'card_number': '2345 6789 0123 4567'}],
  'credit_card_list': [{'card_holder': 'John Doe',
    'card_number': '3456 7890 1234 5678',
    'credit_limit': 10000,
    'cvv': '789',
    'expiry_date': '12/25'}],
  'age': 30},
 {'credit_card_list': [{'cvv': '567',
    'expiry_date': '12/26',
    'card_holder': 'Jane Doe',
    'credit_limit': 20000,
    'card_number': '5678 9012 3456 7890'},
   {'expiry_date': '12/27',
    'cvv': '678',
    'card_number': '6789 0123 4567 8901',
    'credit_limit': 30000,
    'card_holder': 'Jane Doe'}],
  'age': 25,
  'debit_card_list': [{'card_holder': 'Jane Doe',
    'cvv': '234',
    'card_number': '4567 8901 2345 6789',
    'expiry_date': '12/24'}],
  'name': 'Jane Doe'})